In [ ]:
%pylab inline
from statsmodels.kernel_methods.kde_utils import Grid, GridInterpolator
from scipy import interpolate

# Testing 1D interpolation

In [ ]:
ms = ogrid[2:2+2*pi:2**20*1j]
gr = Grid(ms, dtype=float, bin_type='B')
print(gr)
vals = 2+cos(ms)

In [ ]:
interp = GridInterpolator(gr, vals)

In [ ]:
interp([3,4,5])

In [ ]:
xs = r_[1.8:2.1+2*pi:2**20*1j]
ys = interp(xs)
xs.min(), xs.max(), ys.min(), ys.max()

In [ ]:
plot(xs[::512], ys[::512], label='interpolated')
plot(xs[::512], np.interp(xs[::512], gr.full(), vals), label='numpy', color='red', linewidth=2, linestyle='--')
legend(loc='best')

In [ ]:
%timeit interp(xs)

In [ ]:
%timeit np.interp(xs, gr.full(), vals)

### Cyclic interpolation

In [ ]:
gr_cyc = Grid(ms, dtype=float, bin_type='C')
print(gr_cyc)

In [ ]:
xs = r_[2-pi:2+6*pi:2**20*1j]
interp_cyc = GridInterpolator(gr_cyc, vals)

In [ ]:
ys_cyc = interp_cyc(xs)

In [ ]:
plot(xs[::512], ys_cyc[::512], label='interpolated')
axvline(gr_cyc.bounds[0,0], color='r', linestyle='--')
axvline(gr_cyc.bounds[0,1], color='r', linestyle='--')
legend(loc='best')

In [ ]:
%timeit interp_cyc(xs)

### Reflective bounds

In [ ]:
gr_ref = Grid(ms, dtype=float, bin_type='R')
print(gr_ref)

In [ ]:
interp_ref = GridInterpolator(gr_ref, vals)

In [ ]:
ys_ref = interp_ref(xs)

In [ ]:
plot(xs[::512], ys_ref[::512], label='interpolated')
axvline(gr_ref.bounds[0,0], color='r', linestyle='--')
axvline(gr_ref.bounds[0,1], color='r', linestyle='--')
legend(loc='best')

In [ ]:
%timeit interp_ref(xs)

# Testing 2D interpolation

Set the next block to code to evaluate only this section:

In [ ]:
%pylab inline
from statsmodels.kernel_methods.kde_utils import Grid, GridInterpolator
from scipy import interpolate

In [ ]:
ms = ogrid[2*pi/7:pi+2*pi/7:256j, pi*5/7:5/7*pi+pi:256j]
gr = Grid.fromSparse(ms)
npts = prod(gr.shape)
xxs = gr.linear()
gr, xxs.shape

In [ ]:
vals = cos(4*gr[0][:,None]) * sin(6*gr[1])

In [ ]:
interp2d = GridInterpolator(gr, vals)

In [ ]:
lin_grid = gr.full().reshape(npts, 2)
lin_vals = vals.reshape(npts)
interp_test = interpolate.LinearNDInterpolator(lin_grid, lin_vals)

In [ ]:
xs = concatenate([c_[2*pi/7 + pi*rand(1024), 5/7*pi+pi*rand(1024)], xxs], axis=0)
expected = interp_test(xs)
computed = interp2d(xs)

In [ ]:
sqrt(sum((computed - expected)**2))

In [ ]:
f = figure()
ax1 = f.add_subplot(1,3,1)
tripcolor(xs[:,0], xs[:,1], expected, shading='gouraud')
ax2 = f.add_subplot(1,3,2)
tripcolor(xs[:,0], xs[:,1], computed, shading='gouraud')
ax3 = f.add_subplot(1,3,3)
tripcolor(xxs[:,0], xxs[:,1], vals.reshape(npts), shading='gouraud')

### Cyclic Boundaries

In [ ]:
gr_cyc = Grid.fromSparse(ms, bin_type='C')
print(gr_cyc)
interp2d_cyc = GridInterpolator(gr_cyc, vals)
large_grid = Grid.fromSparse(ogrid[-1.5*pi:2*pi:1024j, -pi:3*pi:1024j])

In [ ]:
est_cyc = interp2d_cyc(large_grid.linear())
est_cyc.shape = large_grid.shape

In [ ]:
f = figure()
f.set_size_inches(8, 8)
large_xxs = large_grid.full('C')
pcolormesh(large_xxs[0], large_xxs[1], est_cyc, cmap=cm.jet, shading='gouraud')
xlim(large_grid.bounds[0,0], large_grid.bounds[0,1])
ylim(large_grid.bounds[1,0], large_grid.bounds[1,1])
ax = gca()
rx, ry = gr_cyc.bounds[0,0], gr_cyc.bounds[1,0]
sx = gr_cyc.bounds[0,1] - gr_cyc.bounds[0,0]
sy = gr_cyc.bounds[1,1] - gr_cyc.bounds[1,0]
ax.add_patch(Rectangle((rx, ry), sx, sy, edgecolor='lightgrey', linewidth=2, facecolor='None'))

### Reflected Boundaries

In [ ]:
gr_ref = Grid.fromSparse(ms, bin_type='R')
print(gr_ref)
interp2d_ref = GridInterpolator(gr_ref, vals)

In [ ]:
est_ref = interp2d_ref(large_grid.linear())
est_ref.shape = large_grid.shape

In [ ]:
f = figure()
f.set_size_inches(8, 8)
large_xxs = large_grid.full('C')
pcolormesh(large_xxs[0], large_xxs[1], est_ref, cmap=cm.jet, shading='gouraud')
ax = gca()
rx, ry = gr_ref.bounds[0,0], gr_ref.bounds[1,0]
sx = gr_ref.bounds[0,1] - gr_ref.bounds[0,0]
sy = gr_ref.bounds[1,1] - gr_ref.bounds[1,0]
ax.add_patch(Rectangle((rx, ry), sx, sy, edgecolor='lightgrey', linewidth=2, facecolor='None'))
xlim(large_grid.bounds[0,0], large_grid.bounds[0,1])
ylim(large_grid.bounds[1,0], large_grid.bounds[1,1])

### And finally, mixed

In [ ]:
gr_mix = Grid.fromSparse(ms, bin_type='CR')
print(gr_mix)
interp2d_mix = GridInterpolator(gr_mix, vals)

In [ ]:
est_mix = interp2d_mix(large_grid.linear())
est_mix.shape = large_grid.shape

In [ ]:
f = figure()
f.set_size_inches(8, 8)
large_xxs = large_grid.full('C')
pcolormesh(large_xxs[0], large_xxs[1], est_mix, cmap=cm.jet, shading='gouraud')
ax = gca()
rx, ry = gr_mix.bounds[0,0], gr_mix.bounds[1,0]
sx = gr_mix.bounds[0,1] - gr_mix.bounds[0,0]
sy = gr_mix.bounds[1,1] - gr_mix.bounds[1,0]
ax.add_patch(Rectangle((rx, ry), sx, sy, edgecolor='lightgrey', linewidth=2, facecolor='None'))
xlim(large_grid.bounds[0,0], large_grid.bounds[0,1])
ylim(large_grid.bounds[1,0], large_grid.bounds[1,1])

### Second mix

In [ ]:
gr_mix2 = Grid.fromSparse(ms, bin_type='BR')
print(gr_mix2)
interp2d_mix2 = GridInterpolator(gr_mix2, vals)

In [ ]:
est_mix2 = interp2d_mix2(large_grid.linear())
est_mix2.shape = large_grid.shape

In [ ]:
f = figure()
f.set_size_inches(8, 8)
large_xxs = large_grid.full('C')
pcolormesh(large_xxs[0], large_xxs[1], est_mix2, cmap=cm.jet, shading='gouraud')
ax = gca()
rx, ry = gr_mix2.bounds[0,0], gr_mix2.bounds[1,0]
sx = gr_mix2.bounds[0,1] - gr_mix2.bounds[0,0]
sy = gr_mix2.bounds[1,1] - gr_mix2.bounds[1,0]
ax.add_patch(Rectangle((rx, ry), sx, sy, edgecolor='lightgrey', linewidth=2, facecolor='None'))
xlim(large_grid.bounds[0,0], large_grid.bounds[0,1])
ylim(large_grid.bounds[1,0], large_grid.bounds[1,1])